# DataFrameIt - Exemplo 04: Processamento Incremental com Resume

Este notebook demonstra como usar `resume=True` para processar datasets grandes de forma incremental, com capacidade de pausar e continuar.

**Conceitos demonstrados:**
- Uso de `resume=True`
- Salvamento de progresso parcial
- Continuação após interrupção
- Monitoramento de progresso
- Estratégias para datasets grandes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bdcdo/dataframeit/blob/main/example/03_resume.ipynb)

## 1. Instalação

In [ ]:
!pip install -q dataframeit[google] openpyxl

## 2. Configuração da API Key

In [ ]:
import os

try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    print("API Key carregada dos Secrets do Colab")
except:
    pass

# os.environ['GOOGLE_API_KEY'] = 'sua-chave-aqui'

if 'GOOGLE_API_KEY' not in os.environ:
    print("AVISO: Configure sua GOOGLE_API_KEY antes de continuar")
else:
    print("API Key configurada com sucesso!")

## 3. Imports

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import pandas as pd
from dataframeit import dataframeit
import os

## 4. Definir Modelo Pydantic

In [ ]:
class EmailClassification(BaseModel):
    """Estrutura para classificação de emails."""

    tipo: Literal['trabalho', 'pessoal', 'spam', 'newsletter', 'urgente'] = Field(
        ...,
        description="Tipo/categoria do email"
    )

    prioridade: Literal['alta', 'media', 'baixa'] = Field(
        ...,
        description="Prioridade do email"
    )

    requer_resposta: Literal['sim', 'nao', 'opcional'] = Field(
        ...,
        description="Se o email requer resposta"
    )

## 5. Definir Template

In [ ]:
TEMPLATE = """
Classifique o email abaixo.

Email:
{texto}

Seja preciso na classificação.
"""

## 6. Criar Dataset Simulado

In [ ]:
# Simulamos 20 emails para demonstrar processamento incremental
emails = [
    "Reunião de planejamento do Q4 amanhã às 14h. Presença obrigatória.",
    "Oi! Vamos jantar no sábado?",
    "PROMOÇÃO IMPERDÍVEL! 50% de desconto em todos os produtos!",
    "Seu relatório mensal está pronto para revisão.",
    "Parabéns! Você ganhou um iPhone! Clique aqui para resgatar.",
    "Newsletter semanal: As principais notícias de tecnologia",
    "URGENTE: Sistema fora do ar - necessária ação imediata",
    "Lembrete: Aniversário da Maria é amanhã",
    "Proposta comercial anexa para sua análise",
    "Deseja renovar sua assinatura? Desconto especial!",
    "Ata da última reunião para aprovação",
    "Fotos do fim de semana que você pediu",
    "Boleto com vencimento hoje - evite multa",
    "Novo episódio do podcast que você segue",
    "Preciso da sua aprovação urgente no documento",
    "Convite: Churrasco no domingo às 15h",
    "Seu pedido #12345 foi enviado",
    "Comentários na sua publicação do blog",
    "Atualização de política de privacidade",
    "Feliz aniversário! Desejo tudo de bom!"
]

dados = {
    'email_id': [f'E{i:03d}' for i in range(1, len(emails) + 1)],
    'texto': emails
}

df_original = pd.DataFrame(dados)
print(f"Dataset: {len(df_original)} emails para processar")
df_original.head()

## 7. Cenário 1: Processamento Completo com Resume

In [ ]:
print("Iniciando processamento com resume=True...")

# Processar com resume ativado
df_resultado = dataframeit(
    df_original,
    EmailClassification,
    TEMPLATE,
    text_column='texto',
    resume=True  # Permite continuar de onde parou
)

print("Processamento completo concluído!")

## 8. Análise do Progresso

In [ ]:
status_counts = df_resultado['_dataframeit_status'].value_counts()
total = len(df_resultado)

processados = status_counts.get('processed', 0)
com_erro = status_counts.get('error', 0)
nao_processados = df_resultado['_dataframeit_status'].isna().sum()

print(f"Processados: {processados}/{total} ({processados/total*100:.1f}%)")
print(f"Com erro: {com_erro}/{total}")
print(f"Não processados: {nao_processados}/{total}")

## 9. Visualizar Resultados

In [ ]:
emails_processados = df_resultado[df_resultado['_dataframeit_status'] == 'processed']

if len(emails_processados) > 0:
    print("Primeiros 5 resultados:")
    colunas = ['email_id', 'tipo', 'prioridade', 'requer_resposta']
    display(emails_processados[colunas].head())

    print("\nDistribuição por tipo:")
    print(emails_processados['tipo'].value_counts())

    print("\nDistribuição por prioridade:")
    print(emails_processados['prioridade'].value_counts())

## 10. Salvar Progresso

Salve o progresso para poder continuar depois se necessário.

In [ ]:
# Salvar progresso
arquivo_parcial = 'emails_processados.xlsx'
df_resultado.to_excel(arquivo_parcial, index=False)
print(f"Progresso salvo em: {arquivo_parcial}")

## 11. Cenário 2: Retomando Processamento Interrompido

Se o processamento for interrompido, você pode continuar de onde parou.

In [ ]:
# Simular retomada de processamento
print("Simulando retomada de processamento...")

# Carregar arquivo salvo
df_retomado = pd.read_excel(arquivo_parcial)

# Resume vai processar apenas as linhas não processadas!
df_completo = dataframeit(
    df_retomado,
    EmailClassification,
    TEMPLATE,
    text_column='texto',
    resume=True  # Continua de onde parou!
)

print("Processamento retomado e concluído!")

## 12. Estratégias para Datasets Grandes

### Processamento em Blocos
```python
# Processar em blocos de 100 linhas
for i in range(0, len(df), 100):
    bloco = df[i:i+100]
    resultado = dataframeit(bloco, Model, TEMPLATE, resume=True)
    resultado.to_excel(f'checkpoint_{i}.xlsx', index=False)
```

### Monitoramento de Progresso
```python
# Verificar progresso periodicamente
df_resultado.to_excel('backup_temp.xlsx', index=False)

# Checar quantos faltam
faltam = df_resultado['_dataframeit_status'].isna().sum()
print(f"Faltam {faltam} linhas")
```

### Recuperação de Falhas
```python
# Se houver falha, apenas recarregue e continue
df = pd.read_excel('backup_temp.xlsx')
df = dataframeit(df, Model, TEMPLATE, resume=True)
```

## 13. Boas Práticas

**Recomendações:**
- SEMPRE use `resume=True` para datasets médios/grandes
- Salve progresso periodicamente (.xlsx ou .csv)
- Monitore `_dataframeit_status` para ver progresso
- Use nomes de arquivo com timestamp para backups
- Processe em blocos para datasets muito grandes (>1000 linhas)
- Tenha uma estratégia de backup antes de começar
- Teste com subset pequeno antes do dataset completo

## 14. Limpeza

In [ ]:
# Remover arquivos temporários
if os.path.exists(arquivo_parcial):
    os.remove(arquivo_parcial)
    print(f"Removido: {arquivo_parcial}")

---

## Próximos Passos

- [08_rate_limiting.ipynb](08_rate_limiting.ipynb) - Configurar rate limiting
- [05_advanced_legal.ipynb](05_advanced_legal.ipynb) - Exemplo avançado